<a href="https://colab.research.google.com/github/bgavinash/ClipBoardExample/blob/master/web_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#HTTP request handler class

In [0]:
USE_REQUESTS_LIBRARY = True

In [0]:

class HTTP_request_handler:
  def __init__(self):
    self.session_requests = None
    self.requests_lib_instance = HTTP_handler_requests_library()
    self.custom_instance = HTTP_request_handler_Custom()
    pass
  
  def getPage(self,url):
    if USE_REQUESTS_LIBRARY:
      return self.requests_lib_instance.getPage(url)
    else:
      return self.custom_instance.getPage(url)
  
  def getPageForSession(self,url):
    if USE_REQUESTS_LIBRARY:
      return self.requests_lib_instance.getPageForSession(url)
    else:
      return self.custom_instance.getPageForSession(url)

  def login(self,username,password,url):
    if USE_REQUESTS_LIBRARY:
      self.requests_lib_instance.login(username,password,url)
    else:
      self.custom_instance.login(username,password,url)
    


In [0]:
import requests 
class HTTP_handler_requests_library:
  def __init__(self):
    self.session_requests = None
    pass
  def getPage(self,url):
    url = 'http://cs5700f16.ccs.neu.edu'+ url
    r = requests.get(url)
    return r.content
  
  def getPageForSession(self,url):
    url = 'http://cs5700f16.ccs.neu.edu'+ url
    r = self.session_requests.get(
	  url, 
	  headers = dict(referer = url)
    )
    while not r.ok:
      print('-----retrying-----',r.status_code)
      r = self.session_requests.get(
      url, 
      headers = dict(referer = url)
      )

    return r.content

  def login(self,username,password,url):
    print('Login with :',username,password,url)
    self.session_requests = requests.session()
    result = self.session_requests.get(url)
    
    parser = HTML_parser()
    parser.setPage(result.content,'')
    mw_key = parser.getMiddlewareToken()
    

    payload = {
      "username": username, 
      "password": password, 
      "csrfmiddlewaretoken": mw_key
    }
    result = self.session_requests.post(
      url, 
      data = payload, 
      headers = dict(referer=url)
    )
    print(result.ok)

In [0]:
class HTTP_request_handler_Custom:
  def __init__(self):
    self.session_requests = None
    pass
  def getPage(self,url):
    url = 'http://cs5700f16.ccs.neu.edu'+ url
    r = requests.get(url)
    return r.content
  
  def getPageForSession(self,url):
    url = 'http://cs5700f16.ccs.neu.edu'+ url
    r = self.session_requests.get(
	  url, 
	  headers = dict(referer = url)
    )
    while not r.ok:
      print('-----retrying-----',r.status_code)
      r = self.session_requests.get(
      url, 
      headers = dict(referer = url)
      )

    return r.content

  def login(self,username,password,url):
    print('Login with custom instance :',username,password,url)
    self.session_requests = requests.session()
    result = self.session_requests.get(url)
    
    parser = HTML_parser()
    parser.setPage(result.content,'')
    mw_key = parser.getMiddlewareToken()
    print('mw key is ', mw_key)

    payload = {
      "username": username, 
      "password": password, 
      "csrfmiddlewaretoken": mw_key
    }
    result = self.session_requests.post(
      url, 
      data = payload, 
      headers = dict(referer=url)
    )
    print(result.ok)

#login Module

In [0]:
class Login:
  def __init__(self):
    pass
  
  def loginToFakeBook(self,user,password):
    url = "http://cs5700f16.ccs.neu.edu/accounts/login/"
    req_handler = HTTP_request_handler()
    req_handler.login(user,password,url)
    return req_handler
    


#HTML parser class

In [0]:
from html.parser import HTMLParser

class HTML_parser(HTMLParser):
   
    def getAllLinks(self):
      return self.all_links

    def getSecretKey(self):
      return self.secret_key     

    def getMiddlewareToken(self):
      return self.csfrtoken
    
    def getSessionId(self):
      return self.session_id
    
    def setPage(self,data,url=''):
      self.all_links = set()
      self.session_id = None
      self.csfrtoken = None
      self.secret_key = None
      self.feed((data.decode("utf-8")))
    
    def isFakeBookLink(self,url):
      return 'fakebook' in url

    def parseSessionId(self,text):
      lines = text.split('\n')
      for line in lines:
        startIndex = line.find('sessionid')
        if  startIndex != -1:
          startIndex = line.find('=')
          endIndex = line.find(';')
          self.session_id = line[startIndex+1:endIndex]

    def parseCsfrtoken(self,text):
      lines = text.split('\n')
      for line in lines:
        startIndex = line.find('csrftoken')
        if  startIndex != -1:
          startIndex = line.find('=')
          endIndex = line.find(';')
          print('mw_token in parser is ', self.csfrtoken)
          self.csfrtoken = line[startIndex+1:endIndex]

    def updateAllLinks(self,attrs):
      for name, value in attrs:
        if name == "href" and self.isFakeBookLink(value):
          self.all_links.add(value)
          
    def handle_starttag(self, tag, attrs):
      csfrfound = False
      if tag == 'a':
        self.updateAllLinks(attrs)
      if tag == 'input':
        if not self.csfrtoken:
          for name, value in attrs:
            print('name : ',name,'val :',value)
            if value == 'csrfmiddlewaretoken':
              csfrfound = True
            elif csfrfound and name == 'value':
              self.csfrtoken = value

    def handle_data(self, data):
        if not self.session_id:
          self.parseSessionId(data)
        if not self.csfrtoken:
          self.parseCsfrtoken(data)
        if not self.secret_key:
          if data.find('FLAG: ')!= -1:
            print('Flag found !!! ',data)
            self.secret_key = data

#Crawler class

In [0]:
max_keys_to_find = 1
class Crawler:
  def __init__(self,url,reqh):
    self.links = dict()
    self.links[url] = False
    self.reqh = reqh
  
  def addToLinks(self,newLinks):
    for url in  newLinks:
      if url not in self.links:
        self.links[url.strip()] = False

  def startCrawling(self):
    '''
    Will crawl through all available links and  returns the list of secret keys
    '''
    secret_keys = []
    remaining_links = []
    parser = HTML_parser()
    count = 0
    while(True):
      remainingLinks = [url for url,visited in self.links.items() if not visited ]
      if not remainingLinks:
        print('done with crawling')
        return secret_keys
      for url in remainingLinks :
        self.links[url] = True
        page = self.reqh.getPageForSession(url)
        parser.setPage(page,'')
        key = parser.getSecretKey()
        if key != None:
          print('hurray.... key found at',url,key,len(secret_keys))
          tk.log()
          secret_keys.append(key)
          if len(secret_keys)==max_keys_to_find:
            return secret_keys
        newlinks = parser.getAllLinks()
        self.addToLinks(newlinks)
        count+=1
      #   break # remove this later 
      # break # remove this later for parsing



In [0]:
import time
class TimeKeeper:
  def __init__(self):
    self.startTime = time.time()
  def start(self):
    self.startTime = time.time()
  def log(self):
    print(time.time() - self.startTime )


In [0]:
tk =TimeKeeper()

tk.start()
tk.log()

3.266334533691406e-05


#Driver Code

In [0]:
def loginToFakeBook(user,password):
  url = "http://cs5700f16.ccs.neu.edu/accounts/login/"
  req_handler = HTTP_request_handler()
  req_handler.login(user,password,url)
  return req_handler

In [0]:
tk.start()
session_req_handler = loginToFakeBook('001400450','R26KTI1B')
crawler = Crawler("/fakebook",session_req_handler)
secret_keys = crawler.startCrawling()
tk.log()

Login with : 001400450 R26KTI1B http://cs5700f16.ccs.neu.edu/accounts/login/
name :  id val : id_username
name :  type val : text
name :  name val : username
name :  maxlength val : 30
name :  id val : id_password
name :  type val : password
name :  name val : password
name :  maxlength val : 4096
name :  type val : hidden
name :  name val : csrfmiddlewaretoken
name :  value val : 9bd867a29ca03ee9b69e46f6eb9458be
True
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
-----retrying----- 500
---

In [0]:
secret_keys

['FLAG: 5d62e58ccecda834f295c9b9eac41fb84d9edea950cffde25910811a0bf4cc0c']